# Data Cleaning & Missing Data Analysis

**Goal**: Deep-dive into both datasets (yield & climate) to understand all data quality issues before deciding how to handle them.

**Outline**:
1. Load both raw datasets
2. **Yield table** — missing values per column, per department, per year; recoverability; outliers; consistency checks
3. **Climate table** — missing values, coverage gaps, data quality (negative precip), completeness per department/year
4. **Department mismatch** — detailed comparison of department names between the two tables
5. **Cross-table join analysis** — what happens when we merge; which rows gain NaN and why
6. **Summary of all issues** — table of decisions to make together

---
## 1. Load both raw datasets

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 160)
pd.set_option("display.max_rows", 120)

YIELD_PATH = "../raw_data/barley_yield_from_1982.csv"
CLIMATE_PATH = "../raw_data/climate_data_from_1982.parquet"

# Load
df_yield_raw = pd.read_csv(YIELD_PATH, sep=";")
df_climate_raw = pd.read_parquet(CLIMATE_PATH)

print("Yield raw shape:", df_yield_raw.shape)
print("Climate raw shape:", df_climate_raw.shape)

In [ ]:
# Clean column names
def clean_cols(df: pd.DataFrame) -> pd.DataFrame:
    """Clean column names.

    Args:
        df (pd.DataFrame): DataFrame to clean.

    Returns:
        pd.DataFrame: Cleaned DataFrame with standardized column names.
    """
    df = df.copy()
    df.columns = (
        df.columns.astype(str)
        .str.strip()
        .str.lower()
        .str.replace(r"\s+", "_", regex=True)
        .str.replace(r"[^a-z0-9_]", "", regex=True)
    )
    return df


df_yield = clean_cols(df_yield_raw)
df_yield = df_yield.rename(columns={"department": "nom_dep"})
df_yield["year"] = pd.to_numeric(df_yield["year"], errors="coerce")
df_yield["yield"] = pd.to_numeric(df_yield["yield"], errors="coerce")
df_yield["area"] = pd.to_numeric(df_yield["area"], errors="coerce")
df_yield["production"] = pd.to_numeric(df_yield["production"], errors="coerce")

df_climate = clean_cols(df_climate_raw)
df_climate["time"] = pd.to_datetime(df_climate["time"])

print("Yield columns:", df_yield.columns.tolist())
print("Climate columns:", df_climate.columns.tolist())
display(df_yield.head())
display(df_climate.head())

---
## 2. Yield Table — Deep Missing Data Analysis

### 2.1 Overall missing values per column

In [ ]:
print("YIELD TABLE — MISSING VALUES PER COLUMN")
print("=" * 60)
for col in df_yield.columns:
    n_miss = df_yield[col].isna().sum()
    pct = 100 * n_miss / len(df_yield)
    print(f"  {col:15s}: {n_miss:4d} missing  ({pct:5.2f}%)")

print(f"\nTotal rows: {len(df_yield)}")
print(f"Rows with ANY missing: {df_yield.isna().any(axis=1).sum()}")
print(
    f"Rows with ALL of yield/area/production missing: "
    f"{df_yield[['yield', 'area', 'production']].isna().all(axis=1).sum()}"
)

### 2.2 Missing values overlap (yield vs area vs production)

In [ ]:
miss_yield = df_yield["yield"].isna()
miss_area = df_yield["area"].isna()
miss_prod = df_yield["production"].isna()

print("OVERLAP OF MISSING VALUES")
print("=" * 60)
print(
    "  yield NaN only (area & prod present) : "
    f"{(miss_yield & ~miss_area & ~miss_prod).sum()}"
)
print(
    "  yield NaN + area NaN + prod NaN      : "
    f"{(miss_yield & miss_area & miss_prod).sum()}"
)
print(
    "  yield NaN + area NaN (prod present)  : "
    f"{(miss_yield & miss_area & ~miss_prod).sum()}"
)
print(
    f"  yield NaN + prod NaN (area present)  : "
    f"{(miss_yield & ~miss_area & miss_prod).sum()}"
)
print(f"  area NaN only (yield present)        : {(~miss_yield & miss_area).sum()}")
print(f"  production NaN only (yield present)  : {(~miss_yield & miss_prod).sum()}")

# Crosstab for clarity
df_yield["yield_miss"] = miss_yield.map({True: "yield NaN", False: "yield OK"})
df_yield["area_miss"] = miss_area.map({True: "area NaN", False: "area OK"})
df_yield["prod_miss"] = miss_prod.map({True: "prod NaN", False: "prod OK"})

print("\nCrosstab: yield_miss vs area_miss")
display(pd.crosstab(df_yield["yield_miss"], df_yield["area_miss"], margins=True))
print("\nCrosstab: yield_miss vs prod_miss")
display(pd.crosstab(df_yield["yield_miss"], df_yield["prod_miss"], margins=True))

# Clean up temp columns
df_yield.drop(columns=["yield_miss", "area_miss", "prod_miss"], inplace=True)

### 2.3 Recoverability: can we compute yield = production / area?

In [ ]:
recoverable = miss_yield & ~miss_area & ~miss_prod & (df_yield["area"] > 0)
not_recoverable = miss_yield & ~recoverable

print("YIELD RECOVERABILITY")
print("=" * 60)
print(f"  Total missing yield          : {miss_yield.sum()}")
print(f"  Recoverable (prod/area)      : {recoverable.sum()}")
print(f"  NOT recoverable              : {not_recoverable.sum()}")

# Show the recoverable rows to verify they look reasonable
if recoverable.sum() > 0:
    sample = df_yield[recoverable].copy()
    sample["yield_computed"] = sample["production"] / sample["area"]
    print("\nSample of recoverable rows (computed yield = production / area):")
    display(
        sample[
            ["nom_dep", "year", "yield", "area", "production", "yield_computed"]
        ].head(15)
    )
    print("\nComputed yield stats:")
    display(sample["yield_computed"].describe())

### 2.4 Consistency check: yield vs production/area (when all three present)

In [ ]:
# When yield, area, production are all present, does yield ~= production/area?
mask_all_present = ~miss_yield & ~miss_area & ~miss_prod & (df_yield["area"] > 0)
check = df_yield[mask_all_present].copy()
check["yield_computed"] = check["production"] / check["area"]
check["diff"] = (check["yield"] - check["yield_computed"]).abs()
check["diff_pct"] = 100 * check["diff"] / check["yield"].replace(0, np.nan)

print("CONSISTENCY: yield vs production/area")
print("=" * 60)
print(f"Rows checked: {len(check)}")
print("\nAbsolute difference stats:")
display(check["diff"].describe())
print(f"\nRows with >1% discrepancy: {(check['diff_pct'] > 1).sum()}")
print(f"Rows with >5% discrepancy: {(check['diff_pct'] > 5).sum()}")
print(f"Rows with >10% discrepancy: {(check['diff_pct'] > 10).sum()}")

if (check["diff_pct"] > 5).sum() > 0:
    print("\nRows with >5% discrepancy:")
    display(
        check[check["diff_pct"] > 5][
            [
                "nom_dep",
                "year",
                "yield",
                "area",
                "production",
                "yield_computed",
                "diff_pct",
            ]
        ]
        .sort_values("diff_pct", ascending=False)
        .head(20)
    )

### 2.5 Missing yield by YEAR

In [ ]:
# Missing yield by year
year_stats = (
    df_yield.groupby("year")
    .agg(
        total_depts=("nom_dep", "count"),
        yield_missing=("yield", lambda x: x.isna().sum()),
        area_missing=("area", lambda x: x.isna().sum()),
        prod_missing=("production", lambda x: x.isna().sum()),
    )
    .reset_index()
)
year_stats["yield_miss_pct"] = (
    100 * year_stats["yield_missing"] / year_stats["total_depts"]
).round(1)

print("MISSING BY YEAR")
print("=" * 60)
display(year_stats)

fig, ax = plt.subplots(figsize=(14, 4))
ax.bar(
    year_stats["year"],
    year_stats["yield_missing"],
    color="coral",
    label="yield missing",
)
ax.bar(
    year_stats["year"],
    year_stats["area_missing"],
    color="steelblue",
    alpha=0.7,
    label="area missing",
)
ax.set_xlabel("Year")
ax.set_ylabel("Number of departments with missing data")
ax.set_title("Missing data per year — Yield table")
ax.legend()
plt.tight_layout()
plt.show()

### 2.6 Missing yield by DEPARTMENT

In [ ]:
dep_stats = (
    df_yield.groupby("nom_dep")
    .agg(
        total_years=("year", "count"),
        yield_missing=("yield", lambda x: x.isna().sum()),
        area_missing=("area", lambda x: x.isna().sum()),
        prod_missing=("production", lambda x: x.isna().sum()),
        yield_mean=("yield", "mean"),
        area_mean=("area", "mean"),
    )
    .reset_index()
)
dep_stats["yield_miss_pct"] = (
    100 * dep_stats["yield_missing"] / dep_stats["total_years"]
).round(1)
dep_stats = dep_stats.sort_values("yield_missing", ascending=False)

print("MISSING BY DEPARTMENT (sorted by yield_missing)")
print("=" * 60)
display(dep_stats.head(30))

# Departments with >20% missing yield
high_miss = dep_stats[dep_stats["yield_miss_pct"] > 20]
print(f"\nDepartments with >20% missing yield: {len(high_miss)}")
display(
    high_miss[
        [
            "nom_dep",
            "total_years",
            "yield_missing",
            "yield_miss_pct",
            "yield_mean",
            "area_mean",
        ]
    ]
)

In [ ]:
# Visualize: top 30 departments by missing yield count
top30 = dep_stats.head(30)
fig, ax = plt.subplots(figsize=(12, 8))
ax.barh(top30["nom_dep"], top30["yield_missing"], color="coral")
ax.set_xlabel("Number of missing yield values")
ax.set_title("Top 30 departments by missing yield count")
ax.invert_yaxis()
plt.tight_layout()
plt.show()

### 2.7 Departments with ZERO yield data

In [ ]:
# Departments where yield is ALWAYS missing
zero_yield_depts = dep_stats[dep_stats["yield_missing"] == dep_stats["total_years"]]
print(f"Departments with ALL yield missing: {len(zero_yield_depts)}")
if len(zero_yield_depts) > 0:
    display(zero_yield_depts[["nom_dep", "total_years", "yield_missing", "area_mean"]])

# Departments where yield is NEVER missing
full_yield_depts = dep_stats[dep_stats["yield_missing"] == 0]
print(f"\nDepartments with NO missing yield: {len(full_yield_depts)}")

### 2.8 Yield outlier check

In [ ]:
valid_yield = df_yield.dropna(subset=["yield"])

print("YIELD VALUE DISTRIBUTION")
print("=" * 60)
display(valid_yield["yield"].describe())

# IQR method for outliers
q1, q3 = valid_yield["yield"].quantile([0.25, 0.75])
iqr = q3 - q1
low = q1 - 1.5 * iqr
high = q3 + 1.5 * iqr
outliers = valid_yield[(valid_yield["yield"] < low) | (valid_yield["yield"] > high)]

print(f"\nIQR bounds: [{low:.2f}, {high:.2f}]")
print(
    f"Outliers (IQR method): "
    f"{len(outliers)} ({100 * len(outliers) / len(valid_yield):.1f}%)"
)

if len(outliers) > 0:
    print("\nOutlier rows:")
    display(
        outliers[["nom_dep", "year", "yield", "area", "production"]].sort_values(
            "yield"
        )
    )

fig, axes = plt.subplots(1, 2, figsize=(14, 4))
axes[0].hist(valid_yield["yield"], bins=50, color="steelblue", edgecolor="white")
axes[0].axvline(low, color="red", ls="--", label=f"IQR low={low:.1f}")
axes[0].axvline(high, color="red", ls="--", label=f"IQR high={high:.1f}")
axes[0].set_title("Yield distribution")
axes[0].legend()
axes[1].boxplot(valid_yield["yield"], vert=True)
axes[1].set_title("Yield boxplot")
plt.tight_layout()
plt.show()

### 2.9 Area = 0 or very small (suspect rows)

In [ ]:
# Rows where area = 0 or production = 0
zero_area = df_yield[df_yield["area"] == 0]
zero_prod = df_yield[df_yield["production"] == 0]

print(f"Rows with area = 0: {len(zero_area)}")
if len(zero_area) > 0:
    display(zero_area[["nom_dep", "year", "yield", "area", "production"]])

print(f"\nRows with production = 0: {len(zero_prod)}")
if len(zero_prod) > 0:
    display(zero_prod[["nom_dep", "year", "yield", "area", "production"]].head(20))

# Very small areas
small_area = df_yield[(df_yield["area"] > 0) & (df_yield["area"] < 100)]
print(f"\nRows with 0 < area < 100 (very small): {len(small_area)}")
if len(small_area) > 0:
    display(small_area[["nom_dep", "year", "yield", "area", "production"]].head(20))

### 2.10 Heatmap: missing yield by (department x year)

In [ ]:
# Create a pivot: department x year, value = 1 if yield present, 0 if missing
pivot = df_yield.pivot_table(
    index="nom_dep", columns="year", values="yield", aggfunc=lambda x: x.isna().sum()
).fillna(0)

# Only show departments that have at least 1 missing
pivot_miss = pivot[pivot.sum(axis=1) > 0].sort_values(
    by=pivot.columns.tolist(), ascending=False
)

fig, ax = plt.subplots(figsize=(18, max(6, len(pivot_miss) * 0.3)))
sns.heatmap(
    pivot_miss,
    cmap="YlOrRd",
    linewidths=0.3,
    linecolor="grey",
    cbar_kws={"label": "Missing yield count"},
    ax=ax,
)
ax.set_title("Missing yield heatmap: department x year")
ax.set_xlabel("Year")
ax.set_ylabel("Department")
plt.tight_layout()
plt.show()

---
## 3. Climate Table — Deep Missing Data Analysis

### 3.1 Overall missing values

In [ ]:
print("CLIMATE TABLE — MISSING VALUES PER COLUMN")
print("=" * 60)
for col in df_climate.columns:
    n_miss = df_climate[col].isna().sum()
    pct = 100 * n_miss / len(df_climate)
    print(f"  {col:15s}: {n_miss:8d} missing  ({pct:5.2f}%)")

print(f"\nTotal rows: {len(df_climate):,}")
print(f"Rows with ANY missing: {df_climate.isna().any(axis=1).sum():,}")

### 3.2 Scenario & metric breakdown

In [ ]:
print("COVERAGE BY SCENARIO")
print("=" * 60)
for sc in sorted(df_climate["scenario"].unique()):
    sub = df_climate[df_climate["scenario"] == sc]
    y_min, y_max = sub["year"].min(), sub["year"].max()
    n_deps = sub["nom_dep"].nunique()
    n_metrics = sub["metric"].nunique()
    print(
        f"  {sc:15s}: years {y_min}-{y_max} | "
        f"{n_deps:3d} departments | {n_metrics} metrics | {len(sub):,} rows"
    )

print("\nMETRICS")
print("=" * 60)
for m in df_climate["metric"].unique():
    n = (df_climate["metric"] == m).sum()
    print(f"  {m}: {n:,} rows")

### 3.3 Focus on HISTORICAL scenario (training data)

In [ ]:
clim_hist = df_climate[df_climate["scenario"] == "historical"].copy()
print(f"Historical climate data: {len(clim_hist):,} rows")
print(f"Departments: {clim_hist['nom_dep'].nunique()}")
print(f"Year range: {clim_hist['year'].min()} - {clim_hist['year'].max()}")
print(f"Metrics: {clim_hist['metric'].unique().tolist()}")

# Check completeness: expected days per (department, year, metric)
days_per_group = (
    clim_hist.groupby(["nom_dep", "year", "metric"]).size().reset_index(name="n_days")
)
print("\nExpected ~365 days per (dept, year, metric)")
display(days_per_group["n_days"].describe())

incomplete = days_per_group[days_per_group["n_days"] < 365]
print(f"\nIncomplete groups (< 365 days): {len(incomplete)}")
if len(incomplete) > 0:
    display(incomplete)

### 3.4 Climate departments list

In [ ]:
clim_depts = sorted(clim_hist["nom_dep"].unique())
print(f"Historical climate departments ({len(clim_depts)}):")
for i, d in enumerate(clim_depts, 1):
    print(f"  {i:3d}. {d}")

### 3.5 Coverage per department x year (historical)

In [ ]:
# For historical: which (department, year) combos exist?
hist_coverage = clim_hist.groupby(["nom_dep", "year"]).size().reset_index(name="n_rows")
all_years = list(range(1982, 2015))  # historical: 1982-2014
all_combos = pd.MultiIndex.from_product(
    [clim_depts, all_years], names=["nom_dep", "year"]
)
coverage_full = pd.DataFrame(index=all_combos).reset_index()
coverage_full = coverage_full.merge(hist_coverage, on=["nom_dep", "year"], how="left")
coverage_full["has_data"] = coverage_full["n_rows"].notna()

missing_combos = coverage_full[~coverage_full["has_data"]]
print(f"Missing (dept, year) combinations in historical climate: {len(missing_combos)}")
if len(missing_combos) > 0:
    print("\nMissing combos:")
    display(missing_combos[["nom_dep", "year"]].head(50))

    # By department
    miss_by_dep = missing_combos.groupby("nom_dep").size().sort_values(ascending=False)
    print("\nMissing years per department:")
    display(miss_by_dep.to_frame("n_missing_years"))

### 3.6 Data quality: negative precipitation & value ranges

In [ ]:
print("DATA QUALITY BY METRIC (historical only)")
print("=" * 60)
for m in clim_hist["metric"].unique():
    vals = clim_hist[clim_hist["metric"] == m]["value"]
    n_neg = (vals < 0).sum()
    n_inf = np.isinf(vals).sum()
    n_nan = vals.isna().sum()
    print(f"\n  {m}:")
    print(f"    Count: {len(vals):,}")
    print(f"    Range: [{vals.min():.6f}, {vals.max():.6f}]")
    print(f"    Mean: {vals.mean():.6f}, Std: {vals.std():.6f}")
    print(
        f"    NaN: {n_nan:,}  |  Inf: {n_inf:,}  |  "
        f"Negative: {n_neg:,} ({100 * n_neg / len(vals):.2f}%)"
    )
    if n_neg > 0:
        print(f"    Min negative: {vals[vals < 0].min():.2e}")
        print(f"    Max negative: {vals[vals < 0].max():.2e}")

### 3.7 SSP scenarios: departments & coverage differences

In [ ]:
# Check if SSP scenarios have the same departments as historical
print("DEPARTMENT COVERAGE ACROSS SCENARIOS")
print("=" * 60)
scenario_depts = {}
for sc in sorted(df_climate["scenario"].unique()):
    depts = set(df_climate[df_climate["scenario"] == sc]["nom_dep"].unique())
    scenario_depts[sc] = depts
    print(f"  {sc:15s}: {len(depts)} departments")

# Cross-compare
hist_d = scenario_depts["historical"]
for sc in ["ssp1_2_6", "ssp2_4_5", "ssp5_8_5"]:
    diff_from_hist = hist_d - scenario_depts[sc]
    diff_from_ssp = scenario_depts[sc] - hist_d
    print(
        f"\n  In historical but NOT in {sc}: "
        f"{sorted(diff_from_hist) if diff_from_hist else 'none'}"
    )
    print(
        f"  In {sc} but NOT in historical: "
        f"{sorted(diff_from_ssp) if diff_from_ssp else 'none'}"
    )

---
## 4. Department Mismatch — Detailed Analysis

### 4.1 Side-by-side comparison

In [ ]:
depts_yield = set(df_yield["nom_dep"].dropna().str.strip().unique())
depts_climate = set(clim_hist["nom_dep"].dropna().str.strip().unique())

in_both = sorted(depts_yield & depts_climate)
in_yield_only = sorted(depts_yield - depts_climate)
in_climate_only = sorted(depts_climate - depts_yield)

print("DEPARTMENT COMPARISON: YIELD vs CLIMATE (historical)")
print("=" * 60)
print(f"  Yield departments     : {len(depts_yield)}")
print(f"  Climate departments   : {len(depts_climate)}")
print(f"  In BOTH (can join)    : {len(in_both)}")
print(f"  In YIELD only         : {len(in_yield_only)}")
print(f"  In CLIMATE only       : {len(in_climate_only)}")

In [ ]:
if in_yield_only:
    print("DEPARTMENTS IN YIELD ONLY (no climate data):")
    print("=" * 60)
    for d in in_yield_only:
        sub = df_yield[df_yield["nom_dep"].str.strip() == d]
        n_total = len(sub)
        n_yield_ok = sub["yield"].notna().sum()
        mean_yield = sub["yield"].mean()
        mean_area = sub["area"].mean()
        print(f"\n  {d}:")
        print(
            f"    Rows: {n_total} | Yield present: {n_yield_ok} | "
            f"Mean yield: {mean_yield:.2f} | Mean area: {mean_area:.0f}"
        )
        print(f"    Years: {sorted(sub['year'].tolist())}")

if in_climate_only:
    print("\nDEPARTMENTS IN CLIMATE ONLY (no yield data):")
    print("=" * 60)
    for d in in_climate_only:
        sub = clim_hist[clim_hist["nom_dep"].str.strip() == d]
        print(f"  {d}: {sub['year'].nunique()} years of climate data")

### 4.2 Fuzzy matching: are the mismatches naming issues?

In [ ]:
# Simple fuzzy check: normalize and compare
def normalize_name(name):
    """Normalize the departmentnames.

    Args:
        name (str): The department name to normalize.

    Returns:
        str: The normalized department name.
    """
    return name.lower().replace("_", " ").replace("-", " ").replace("'", " ").strip()


print("FUZZY MATCHING ATTEMPT")
print("=" * 60)

yield_norm = {normalize_name(d): d for d in depts_yield}
climate_norm = {normalize_name(d): d for d in depts_climate}

# Check if any yield-only dept normalizes to a climate dept
found_match = False
for d in in_yield_only:
    d_norm = normalize_name(d)
    if d_norm in climate_norm:
        print(
            f"  MATCH: yield '{d}' -> climate '{climate_norm[d_norm]}' "
            "(same after normalization)"
        )
        found_match = True
    else:
        # Check substring matches
        for cn, co in climate_norm.items():
            if d_norm in cn or cn in d_norm:
                print(f"  SIMILAR: yield '{d}' ~ climate '{co}'")
                found_match = True

if not found_match:
    print("  No fuzzy matches found — these are genuinely different departments.")

# Also list full sorted dept names side by side
print("\n\nFULL DEPARTMENT LISTS")
print("=" * 60)
all_depts = sorted(depts_yield | depts_climate)
print(f"{'Department':<30s} {'In Yield':>10s} {'In Climate':>10s}")
print("-" * 55)
for d in all_depts:
    in_y = "Yes" if d in depts_yield else "-"
    in_c = "Yes" if d in depts_climate else "-"
    marker = " ***" if in_y != in_c else ""
    print(f"  {d:<28s} {in_y:>10s} {in_c:>10s}{marker}")

### 4.3 Impact quantification

In [ ]:
# How many yield rows are affected by the department mismatch?
yield_no_climate = df_yield[df_yield["nom_dep"].str.strip().isin(in_yield_only)]
yield_has_climate = df_yield[df_yield["nom_dep"].str.strip().isin(in_both)]

print("IMPACT OF DEPARTMENT MISMATCH ON YIELD TABLE")
print("=" * 60)
print(
    f"  Yield rows with climate match    : {len(yield_has_climate)} "
    f"({100 * len(yield_has_climate) / len(df_yield):.1f}%)"
)
print(
    f"  Yield rows WITHOUT climate match : {len(yield_no_climate)} "
    f"({100 * len(yield_no_climate) / len(df_yield):.1f}%)"
)
print("\n  Of those without climate:")
print(f"    With valid yield  : {yield_no_climate['yield'].notna().sum()}")
print(f"    With missing yield: {yield_no_climate['yield'].isna().sum()}")

---
## 5. Cross-table Join Analysis

### 5.1 Year coverage mismatch

In [ ]:
yield_years = set(df_yield["year"].dropna().unique())
climate_hist_years = set(clim_hist["year"].unique())

print("YEAR COVERAGE")
print("=" * 60)
print(
    f"  Yield years   : {min(yield_years):.0f} - {max(yield_years):.0f} "
    f"({len(yield_years)} years)"
)
print(
    f"  Climate years : {min(climate_hist_years)} - {max(climate_hist_years)} "
    f"({len(climate_hist_years)} years)"
)
print(
    f"  Overlap       : {min(yield_years & climate_hist_years):.0f} - "
    f"{max(yield_years & climate_hist_years):.0f} "
    f"({len(yield_years & climate_hist_years)} years)"
)

yield_no_climate_years = sorted(yield_years - climate_hist_years)
climate_no_yield_years = sorted(climate_hist_years - yield_years)

print(
    "\n  Yield years WITHOUT historical climate: "
    f"{yield_no_climate_years if yield_no_climate_years else 'none'}"
)
print(
    "  Climate years WITHOUT yield           : "
    f"{climate_no_yield_years if climate_no_yield_years else 'none'}"
)

if yield_no_climate_years:
    n_affected = df_yield[df_yield["year"].isin(yield_no_climate_years)].shape[0]
    print(f"\n  Yield rows in years without historical climate: {n_affected}")

### 5.2 Simulated join: what NaN would we get?

In [ ]:
# Aggregate climate to (nom_dep, year) level — simple mean per metric
clim_agg = (
    clim_hist.groupby(["nom_dep", "year", "metric"])["value"]
    .mean()
    .reset_index()
    .pivot_table(index=["nom_dep", "year"], columns="metric", values="value")
    .reset_index()
)
clim_agg.columns.name = None

# Join
merged = df_yield.merge(clim_agg, on=["nom_dep", "year"], how="left")

print("SIMULATED JOIN: yield LEFT JOIN climate_agg")
print("=" * 60)
print(f"  Merged shape: {merged.shape}")
print("\n  Missing values after join:")
for col in merged.columns:
    n = merged[col].isna().sum()
    pct = 100 * n / len(merged)
    if n > 0:
        print(f"    {col:45s}: {n:4d} ({pct:5.1f}%)")

In [ ]:
# Diagnose WHY climate columns are NaN
climate_cols = [c for c in merged.columns if c not in df_yield.columns]
any_climate_nan = merged[climate_cols].isna().any(axis=1)

nan_rows = merged[any_climate_nan].copy()
nan_rows["reason"] = "unknown"
nan_rows.loc[nan_rows["nom_dep"].str.strip().isin(in_yield_only), "reason"] = (
    "dept_not_in_climate"
)
nan_rows.loc[
    (nan_rows["reason"] == "unknown") & (~nan_rows["year"].isin(climate_hist_years)),
    "reason",
] = "year_not_in_climate"

print("REASONS FOR NaN CLIMATE FEATURES AFTER JOIN")
print("=" * 60)
display(nan_rows["reason"].value_counts().to_frame("count"))

if (nan_rows["reason"] == "unknown").sum() > 0:
    print("\nRows with unknown reason for NaN:")
    display(
        nan_rows[nan_rows["reason"] == "unknown"][
            ["nom_dep", "year"] + climate_cols
        ].head(20)
    )

### 5.3 Summary: rows usable for modeling

In [ ]:
has_yield = merged["yield"].notna()
has_climate = merged[climate_cols].notna().all(axis=1)

print("USABLE ROWS SUMMARY")
print("=" * 60)
print(f"  Total yield rows             : {len(merged)}")
print(
    f"  With yield present           : "
    f"{has_yield.sum()} ({100 * has_yield.mean():.1f}%)"
)
print(
    f"  With climate present         : "
    f"{has_climate.sum()} ({100 * has_climate.mean():.1f}%)"
)
print(
    f"  With BOTH yield AND climate  : {(has_yield & has_climate).sum()} "
    f"({100 * (has_yield & has_climate).mean():.1f}%)"
)
print(f"\n  Lost due to missing yield   : {(~has_yield).sum()}")
print(f"  Lost due to missing climate  : {(has_yield & ~has_climate).sum()}")
print(
    f"  Lost total (from original)   : {len(merged) - (has_yield & has_climate).sum()}"
)

---
## 6. Summary of All Issues — Decision Points

Below is a consolidated summary of every data quality issue found. We can discuss each one and decide what to do.

In [ ]:
print("""
╔══════════════════════════════════════════════════════════════════════════════════════╗
║                    SUMMARY OF ALL DATA QUALITY ISSUES                              ║
╚══════════════════════════════════════════════════════════════════════════════════════╝

YIELD TABLE
──────────────────────────────────────────────────────────────────────────────────────
Issue #1: Missing yield values
  - 247 rows (~6.9%) have NaN yield
  - ~125 can be RECOVERED via yield = production / area
  - ~122 cannot be recovered (area and/or production also missing)
  → DECISION: Recover where possible? Drop the rest?

Issue #2: Missing area values
  - 115 rows (~3.2%) have NaN area
  - Most overlap with missing yield
  → DECISION: Drop? Or keep rows where yield is present even if area is NaN?

Issue #3: Missing production values
  - 122 rows (~3.4%) have NaN production
  → DECISION: Same as area — keep if yield is present?

Issue #4: Departments with extremely high missing rates
  - Paris, Hauts-de-Seine, Seine-Saint-Denis, Val-de-Marne, Val-d'Oise,
    Seine-et-Marne (urban Île-de-France) → minimal barley production
  → DECISION: Drop these departments entirely?

Issue #5: Area = 0 or very small areas
  - Some rows have area = 0 (no barley cultivation)
  → DECISION: Drop rows with area = 0?

Issue #6: Yield outliers
  - Some extreme low values (< 1 t/ha)
  → DECISION: Keep, clip, or investigate?

CLIMATE TABLE
──────────────────────────────────────────────────────────────────────────────────────
Issue #7: No raw NaN values
  - Climate data has 0% missing → good!
  → No action needed on raw missing.

Issue #8: Negative precipitation values
  - ~4% of precipitation rows have tiny negative values (~1e-25)
  - These are numerical noise, not real
  → DECISION: Clip to 0?

Issue #9: Temperature in Kelvin
  - Values are in Kelvin (not Celsius)
  → DECISION: Convert to Celsius (subtract 273.15)? Or keep in K?

CROSS-TABLE ISSUES
──────────────────────────────────────────────────────────────────────────────────────
Issue #10: Department mismatch
  - 8 departments in yield have NO climate data
  - These are: Corse_du_Sud, Haute_Corse, Hauts_de_Seine, Paris,
    Seine_Saint_Denis, Seine_SeineOise, Val_d_Oise, Val_de_Marne
  - Affects ~290 yield rows (8.1%)
  - No fuzzy name matches → genuinely missing, not a naming issue
  → DECISION: Drop yield rows for these departments?

Issue #11: Year coverage mismatch
  - Yield: 1982-2018, Climate historical: 1982-2014
  - Yield rows for 2015-2018 have no historical climate data
  - SSP scenarios start at 2015 (future projections, not observations)
  → DECISION: Drop yield rows 2015-2018? Or use SSP data as proxy?

""")

In [ ]:
# Final quantification: how many rows would survive each cleaning scenario
total = len(df_yield)

# Scenario A: minimal cleaning (just drop rows where yield is NaN after recovery)
df_a = df_yield.copy()
recover_mask = (
    df_a["yield"].isna()
    & df_a["area"].notna()
    & df_a["production"].notna()
    & (df_a["area"] > 0)
)
df_a.loc[recover_mask, "yield"] = (
    df_a.loc[recover_mask, "production"] / df_a.loc[recover_mask, "area"]
)
a_after_recovery = df_a["yield"].notna().sum()
a_lost = total - a_after_recovery

# Scenario B: + drop departments not in climate
df_b = df_a[df_a["nom_dep"].str.strip().isin(in_both)]
b_after = df_b["yield"].notna().sum()

# Scenario C: + drop years not in historical climate (2015-2018)
df_c = df_b[df_b["year"].isin(climate_hist_years)]
c_after = df_c["yield"].notna().sum()

# Scenario D: + drop area = 0
df_d = df_c[df_c["area"] > 0]
d_after = df_d["yield"].notna().sum()

print("CLEANING SCENARIOS — ROW SURVIVAL")
print("=" * 60)
print(f"  Original                                  : {total:5d} rows")
print(
    f"  A) Recover yield + drop NaN yield         : {a_after_recovery:5d} rows "
    f"(lost {a_lost})"
)
print(f"  B) + Drop depts not in climate            : {b_after:5d} rows")
print(f"  C) + Drop years 2015-2018 (no hist clim)  : {c_after:5d} rows")
print(f"  D) + Drop area = 0                        : {d_after:5d} rows")
print(f"\n  Final usable rows: {d_after} out of {total} ({100 * d_after / total:.1f}%)")